# 검색성능 최적화를 위한 벡터DB 구축

In [3]:
%pip install -Uq langchain langchain-openai langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")
os.environ['PINECONE_API_KEY'] = os.getenv("pinecone_key")

## Pinecone Index 생성

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone() # pinecone api key 인증
print(pc.list_indexes().names()) # 인덱스 이름 확인


# 인덱스명 ir
if 'ir' not in pc.list_indexes().names():
    pc.create_index(
        name='ir',                # 인덱스 이름
        dimension=1536,           # 임베딩 차원
        metric='cosine',          # 유사도 기준
        spec=ServerlessSpec(
            region='us-east-1',   # 지역 설정
            cloud='aws'           # 클라우드 설정
        )
    )
    print('ir인덱스가 생성되었습니다.')
else:
    print('ir인덱스가 이미 존재합니다.')



['winemag-data-130k-v2', 'pinecone-first', 'ir-multihop']
ir인덱스가 생성되었습니다.


## VectorStore 연결

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding_model = OpenAIEmbeddings()  # OpenAI 임베딩 객체
vector_store = PineconeVectorStore(   # 벡터스토어 연결
    index_name='ir',                  # 연결할 인덱스
    embedding=embedding_model         # 임베딩 모델
)

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Document Upsert

In [7]:
import pandas as pd

df = pd.read_csv('documents.csv')
df

,idx,doc_id,title,content
0,0,D1,제주도 여행 가이드,"제주도는 대한민국의 대표 관광지로서, 한라산 등반, 성산 일출봉 관광, 해변 활동(..."
1,1,D2,전주 비빔밥과 진주 비빔밥 차이점,"비빔밥은 조선 시대부터 전해 내려온 대표적 한국 음식으로, 밥 위에 고명(채소·고기..."
2,2,D3,걸스데이 히트곡 분석,"걸스데이는 2010년 데뷔한 대한민국의 4인조 걸그룹으로, 대표곡으로는 “Somet..."
3,3,D4,세종대왕과 훈민정음,세종대왕(1397~1450)은 훈민정음을 창제하여 한글을 보급한 조선의 4대 임금입...
4,4,D5,이순신 장군의 명량 해전,이순신 장군(1545~1598)은 임진왜란 당시 명량 해전에서 13척의 배로 133...
5,5,D6,2024년 기후 변화 종합 보고서,"2024년 전 지구 평균 기온은 산업화 이전 대비 약 1.2℃ 상승했으며, 해수면 ..."
6,6,D7,AI 기술 동향 및 윤리,"최근 인공지능 분야에서는 생성형 AI, 멀티모달 모델, 강화학습 기반 에이전트 개발..."
7,7,D8,서울 지하철 이용 가이드,"서울 지하철은 1호선부터 9호선까지 운행되며, 주요 환승역으로는 서울역·강남역·종로..."
8,8,D9,판소리 “춘향가” 서사 구조,"판소리는 소리꾼과 고수가 함께 공연하는 한국 전통 음악으로, 대표 작품에는 “춘향가..."
9,9,D10,한국 축구 대표팀 주요 기록,"한국 축구 대표팀은 2002 한일 월드컵 4강 진출, 2012 런던 올림픽 동메달 ..."


In [ ]:
# Document 변환
from langchain_core.documents import Document

docs_to_index = []
for idx, row in df.iterrows():  # df을 행 단위로 순회
    doc_id = row['doc_id']
    content = row['content']
    doc = Document(page_content=content, metadata={'doc_id': doc_id})  # Document 생성
    docs_to_index.append(doc)

print(f'{len(docs_to_index)}개 문서 준비완료')

30개 문서 준비완료


In [9]:
# 문서 업서트
vector_store.add_documents(docs_to_index)

['0fc7d4d9-10b8-46a3-b22c-6a05e9800e3a',
 'c3928189-5635-4e9a-ab8a-d5d2b956aad6',
 'ee8a7fde-52eb-4057-b457-46ee5981830d',
 '7b287b93-bd5b-44b0-a106-8a35bc6283ca',
 '268fda66-4346-46c0-8a12-8c3bafc9c584',
 '4e38ec18-ee8d-4b8f-bcd3-f403c9768b69',
 '7f139a61-a068-451c-b295-770b7d26e452',
 '630f5b30-1d6d-4160-a290-f338eb4c496e',
 '4417227b-37e8-4808-89cd-2abca98e241a',
 'e426b095-5d79-4d04-a378-e71d5ec960b4',
 'be67ba5a-5415-464c-92ea-18e3f204ceca',
 'ad76eed1-c23a-41ff-9796-3e4c00f537ec',
 '988fbfa4-dabd-4b7e-95bf-6e92d8094941',
 '9cfd7679-0383-4495-a6df-fe865618257d',
 '7c951872-26b6-4614-85da-61e857c391cf',
 '1ef86c98-1670-434b-a561-55b3344e8340',
 '0469f447-7970-42f5-a3ab-7ba6f6260d52',
 'eac855c5-7f02-46cd-88c1-ed152bd5499e',
 'b0665030-b7dd-4ddc-b093-69c846b5298c',
 '0adaa21d-ccad-40b5-b224-f0a937e081f4',
 '260e9930-14a5-4152-ac3d-223ee45fb5a0',
 'd35e7ba4-819f-404f-96b1-e93ae8d4a2f9',
 '22a71615-98f5-46a3-a2d3-2e0a8af316c7',
 '1728dab3-f772-45b4-b2e7-42d6f69d8f64',
 '1fe7df9d-497a-

In [ ]:
# 벡터 검색

query = '제주도 여행'

results = vector_store.similarity_search_with_score(query, k=5)  # 가장 유사한 문서 5개 + 점수 반환
for rank, (doc, score) in enumerate(results, 1):  # 순위(1부터), (문서, 점수) 순회
    print(f'{rank}: Score {score}')
    print(f'doc_id: {doc.metadata["doc_id"]}')
    print(f'content: {doc.page_content}')
    print()

1: Score 0.883606
doc_id: D1
content: 제주도는 대한민국의 대표 관광지로서, 한라산 등반, 성산 일출봉 관광, 해변 활동(...

2: Score 0.88079834
doc_id: D12
content: 서울 근교에서 당일치기로 다녀올 만한 여행지로는 가평 쁘띠프랑스, 남양주 수종사, ...

3: Score 0.814819217
doc_id: D8
content: 서울 지하철은 1호선부터 9호선까지 운행되며, 주요 환승역으로는 서울역·강남역·종로...

4: Score 0.811950743
doc_id: D3
content: 걸스데이는 2010년 데뷔한 대한민국의 4인조 걸그룹으로, 대표곡으로는 “Somet...

5: Score 0.804992557
doc_id: D4
content: 세종대왕(1397~1450)은 훈민정음을 창제하여 한글을 보급한 조선의 4대 임금입...

